In [1]:
#Connect to GIS
from arcgis.gis import GIS
gis = GIS("home")

In [2]:
import requests
import pandas as pd
import zipfile
import os
import io
import arcpy
from arcgis import features, GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from arcgis.features.analysis import join_features
from arcgis.features import FeatureLayerCollection, FeatureLayer
from arcgis.geometry import SpatialReference , Geometry
import json

In [3]:
directory= 'C:\Mac\Home\Documents\GIS5571_FinalProjectData'
output_gdb = r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb" 

#Get TIGER gdb
url= r'https://www2.census.gov/geo/tiger/TGRGDB24/tlgdb_2024_a_19_ia.gdb.zip'
request= requests.get(url)
with open('C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb.zip', 'wb') as file:
    file.write(request.content)
    
#unzip
zip_path ='C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(directory)

In [4]:
county= r'C:\Mac\Home\Documents\GIS5571_FinalProjectData\tlgdb_2024_a_19_ia.gdb\County'
arcpy.conversion.FeatureClassToFeatureClass(county, output_gdb, 'county_boundaries')

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\county_boundaries'>

In [5]:
esb_fl = FeatureLayer("https://services.arcgis.com/vPD5PVLI6sfkZ5E4/ArcGIS/rest/services/Electrical_Service_Boundaries/FeatureServer/14")
sedf= pd.DataFrame.spatial.from_layer(esb_fl)

out_name = "Utility"
sedf.spatial.to_featureclass(location=f"{output_gdb}/{out_name}")

'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\Utility'

In [6]:
ec_fl = FeatureLayer("https://arcgis.netl.doe.gov/server/rest/services/Hosted/2024_Coal_Closure_Energy_Communities/FeatureServer/0")
ecsedf= pd.DataFrame.spatial.from_layer(ec_fl)

out_name = "EC2024_all"
ecsedf.spatial.to_featureclass(location=f"{output_gdb}/{out_name}")

'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\EC2024_all'

In [7]:
utility= arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Utility",
    selection_type="NEW_SELECTION",
    where_clause="Owner = 'MidAmerican Energy' Or Owner = 'Interstate Power and Light Company'",
    invert_where_clause=None
)

In [8]:
output_fc = os.path.join(output_gdb, "IOUUtility")
arcpy.CopyFeatures_management(utility, output_fc)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\IOUUtility'>

In [24]:
arcpy.management.Dissolve(
    in_features="IOUUtility",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\IOUUtility_Dissolve",
    dissolve_field="owner",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\IOUUtility_Dissolve'>

In [1]:
arcpy.analysis.Intersect(
    in_features="IOUUtility_Dissolve #;county_boundaries #",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\Utility_AHJ",
    join_attributes="ALL",
    cluster_tolerance=None,
    output_type="INPUT"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\Utility_AHJ'>

In [13]:
# Definition Query for EC: State= IA
aprx = arcpy.mp.ArcGISProject("CURRENT")
map_obj = aprx.listMaps("Map")[0] 
layer = map_obj.listLayers("EC2024_All")[0]
layer.definitionQuery = "state_name = 'Iowa'"  

In [1]:
arcpy.conversion.TableToGeodatabase(
    Input_Table=r"C:\Mac\Home\Documents\GIS5571_FinalProjectData\IowaSiteAnalytics.csv",
    Output_Geodatabase=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb'>

In [13]:
arcpy.management.Dissolve(
    in_features="EC2024_all",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\ec_dissolve",
    dissolve_field=None,
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\ec_dissolve'>

## Boolean Decision Analysis and Adding Weights

In [1]:
#Utility AHJ and EC Union
arcpy.analysis.Union(
    in_features="Utility_AHJ #;ec_dissolve #",
    out_feature_class=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\UtilityAHJ_EC_Union",
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

<Result 'C:\\Mac\\Home\\Documents\\ArcGIS\\Projects\\GIS5571Final_Project\\MyProject2\\MyProject2.gdb\\UtilityAHJ_EC_Union'>

In [15]:
# Add weights field
arcpy.management.AddField(
    in_table="UtilityAHJ_EC_Union",
    field_name="weight",
    field_type="FLOAT",
    field_precision=None,
    field_scale=None,
    field_length=None,
    field_alias="",
    field_is_nullable="NULLABLE",
    field_is_required="NON_REQUIRED",
    field_domain=""
)

<Result 'UtilityAHJ_EC_Union'>

In [3]:
#Select rows with no IOU utility
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = ''",
    invert_where_clause=None
)

<Result 'UtilityAHJ_EC_Union'>

In [4]:
#Delete rows with no IOU utility
arcpy.management.DeleteRows(
    in_rows="UtilityAHJ_EC_Union"
)

<Result 'UtilityAHJ_EC_Union'>

In [18]:
# Assign weights to Utility/EC combinations
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'Interstate Power and Light Company' And name = 'EC2024'",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression="0.33",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [17]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'MidAmerican Energy' And name = 'EC2024'",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression="1",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [19]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'Interstate Power and Light Company' And name = ''",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression="0",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [20]:
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="UtilityAHJ_EC_Union",
    selection_type="NEW_SELECTION",
    where_clause="owner = 'MidAmerican Energy' And name = ''",
    invert_where_clause=None
)

arcpy.management.CalculateField(
    in_table="UtilityAHJ_EC_Union",
    field="weight",
    expression="0.67",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'UtilityAHJ_EC_Union'>

In [ ]:
#weighted overlay
arcpy.conversion.PolygonToRaster(
    in_features="UtilityAHJ_EC_Union",
    value_field="weight",
    out_rasterdataset=r"C:\Mac\Home\Documents\ArcGIS\Projects\GIS5571Final_Project\MyProject2\MyProject2.gdb\UtilityAHJ_EC_Union_PolygonToRaster",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=1400,
    build_rat="BUILD"
)